# 傷病名分類カテゴリ管理ツール

このノートブックでは、傷病名分類のカテゴリと正規表現パターンを管理するための対話的なツールを提供します。

## 準備

まず必要なパッケージをインストールしてください：

```bash
pip install ipywidgets pandas matplotlib seaborn japanize-matplotlib
```

また、拡張機能を有効にしてください：

```bash
jupyter nbextension enable --py widgetsnbextension
```

In [2]:
import json
import os
import re
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import matplotlib.pyplot as plt
import seaborn as sns
import japanize_matplotlib

## カテゴリ管理クラス

傷病名分類のカテゴリを管理するクラスを定義します。

In [3]:
import collections  # ファイルの先頭に追加

class CategoryManager:
    """傷病名分類のカテゴリを管理するクラス"""
    def __init__(self, file_path='disease_categories.json'):
        """初期化"""
        self.file_path = file_path
        self.categories = self.load_categories()
        self._compiled_patterns = {}
        self._compile_patterns()

    def load_categories(self):
        """カテゴリ定義ファイルを読み込む（順序を維持）"""
        try:
            with open(self.file_path, 'r', encoding='utf-8') as f:
                categories = json.load(f, object_pairs_hook=collections.OrderedDict)
            return categories
        except FileNotFoundError:
            print(f"ファイル '{self.file_path}' が見つかりません。空のカテゴリ辞書を作成します。")
            return collections.OrderedDict()
        except json.JSONDecodeError:
            print(f"ファイル '{self.file_path}' の形式が正しくありません。空のカテゴリ辞書を作成します。")
            return collections.OrderedDict()
    
    def save_categories(self):
        """カテゴリ定義ファイルを保存する"""
        with open(self.file_path, 'w', encoding='utf-8') as f:
            json.dump(self.categories, f, ensure_ascii=False, indent=2)
        print(f"カテゴリ定義を '{self.file_path}' に保存しました。")
    
    def _compile_patterns(self):
        """正規表現パターンをコンパイルする"""
        self._compiled_patterns = {}
        for category, pattern in self.categories.items():
            try:
                self._compiled_patterns[category] = re.compile(pattern)
            except re.error as e:
                print(f"カテゴリ '{category}' の正規表現エラー: {e}")
    
    def add_category(self, name, pattern):
        """新しいカテゴリを追加する"""
        try:
            re.compile(pattern)
            self.categories[name] = pattern
            self._compile_patterns()
            return True
        except re.error as e:
            print(f"正規表現エラー: {e}")
            return False
    
    def update_category(self, old_name, new_name, new_pattern=None):
        """カテゴリを更新する"""
        if old_name not in self.categories:
            print(f"カテゴリ '{old_name}' は存在しません。")
            return False
        
        # パターンの更新
        if new_pattern is not None:
            try:
                re.compile(new_pattern)
                pattern = new_pattern
            except re.error as e:
                print(f"正規表現エラー: {e}")
                return False
        else:
            pattern = self.categories[old_name]
        
        # 名前の更新
        if new_name != old_name:
            if new_name in self.categories and new_name != old_name:
                print(f"カテゴリ '{new_name}' は既に存在します。")
                return False
            del self.categories[old_name]
        
        self.categories[new_name] = pattern
        self._compile_patterns()
        return True
    
    def delete_category(self, name):
        """カテゴリを削除する"""
        if name not in self.categories:
            print(f"カテゴリ '{name}' は存在しません。")
            return False
        
        del self.categories[name]
        if name in self._compiled_patterns:
            del self._compiled_patterns[name]
        return True
    
    def add_terms_to_category(self, name, terms):
        """カテゴリに語句を末尾に追加する"""
        if name not in self.categories:
            print(f"カテゴリ '{name}' は存在しません。")
            return False
        
        current_pattern = self.categories[name]
        current_terms = current_pattern.strip("()").split("|")
        
        # 新しい語句を分割
        if isinstance(terms, str):
            new_terms = terms.split("|")
        else:
            new_terms = terms
        
        # 語句を末尾に追加（重複は除去せずそのまま追加）
        all_terms = current_terms.copy()
        for term in new_terms:
            if term not in current_terms:  # 既存の語句と重複しない場合のみ追加
                all_terms.append(term)
        
        new_pattern = "(" + "|".join(all_terms) + ")"
        
        # パターンの形式チェック
        try:
            re.compile(new_pattern)
            self.categories[name] = new_pattern
            self._compile_patterns()
            return True
        except re.error as e:
            print(f"正規表現エラー: {e}")
            return False
    
    def test_pattern(self, text, category=None):
        """テキストがパターンにマッチするかテストする"""
        results = []
        
        if category is not None:
            if category not in self._compiled_patterns:
                return [(category, False, "カテゴリが存在しません")]
            pattern = self._compiled_patterns[category]
            match = pattern.search(text)
            return [(category, match is not None, match.group(0) if match else None)]
        
        for cat, pattern in self._compiled_patterns.items():
            match = pattern.search(text)
            results.append((cat, match is not None, match.group(0) if match else None))
        
        return results
    
    def classify(self, text):
        """テキストを最初にマッチするカテゴリに分類する"""
        for cat, pattern in self._compiled_patterns.items():
            if pattern.search(text):
                return cat
        return "その他"
    
    def export_to_csv(self, file_path):
        """カテゴリとパターンをCSVに出力する"""
        result = []
        for category, pattern in self.categories.items():
            terms = pattern.strip("()").split("|")
            for term in terms:
                result.append({"カテゴリ": category, "語句": term})
        
        df = pd.DataFrame(result)
        df.to_csv(file_path, encoding='utf-8', index=False)
        print(f"カテゴリと語句を '{file_path}' に出力しました。")
        return df
    
    def import_from_csv(self, file_path, overwrite=False):
        """CSVからカテゴリとパターンを読み込む"""
        df = pd.read_csv(file_path, encoding='utf-8')
        
        if "カテゴリ" not in df.columns or "語句" not in df.columns:
            print("CSVファイルには 'カテゴリ' と '語句' の列が必要です。")
            return False
        
        # カテゴリごとに語句をグループ化
        new_categories = {}
        for category, group in df.groupby("カテゴリ"):
            terms = group["語句"].tolist()
            pattern = "(" + "|".join(terms) + ")"
            try:
                re.compile(pattern)
                new_categories[category] = pattern
            except re.error as e:
                print(f"カテゴリ '{category}' の正規表現エラー: {e}")
        
        if overwrite:
            self.categories = new_categories
        else:
            self.categories.update(new_categories)
        
        self._compile_patterns()
        print(f"{len(new_categories)}個のカテゴリを '{file_path}' から読み込みました。")
        return True
    
    def get_category_terms(self, name):
        """カテゴリに含まれる語句のリストを取得する（元の順序を維持）"""
        if name not in self.categories:
            return []
        
        pattern = self.categories[name]
        # 括弧を削除し、パイプで区切って語句のリストを取得
        terms = pattern.strip("()").split("|")
        return terms
    
    def get_category_stats(self):
        """カテゴリごとの語句数などの統計情報を取得する"""
        stats = []
        for category, pattern in self.categories.items():
            terms = pattern.strip("()").split("|")
            stats.append({
                "カテゴリ": category,
                "語句数": len(terms),
                "パターン長": len(pattern)
            })
        
        return pd.DataFrame(stats)

## カテゴリマネージャーのインスタンス化

JSONファイルからカテゴリを読み込み、管理クラスを初期化します。

In [ ]:
import os
import glob
from pathlib import Path

# リポジトリルートを推定（notebookディレクトリの親ディレクトリ）
def find_repo_root():
    current_dir = os.getcwd()
    notebook_dir = Path(current_dir)
    
    # カレントディレクトリがすでにnotebookディレクトリなら
    if notebook_dir.name == 'notebook':
        return str(notebook_dir.parent)
    
    # それ以外の場合は親ディレクトリを探索
    potential_repo_root = notebook_dir
    for _ in range(3):  # 安全のため最大3階層まで探索
        if (potential_repo_root / 'src' / 'bin').exists():
            return str(potential_repo_root)
        potential_repo_root = potential_repo_root.parent
    
    # 見つからない場合はカレントディレクトリを返す
    return current_dir

# リポジトリルートを取得
repo_root = find_repo_root()
json_pattern = os.path.join(repo_root, 'src', 'bin', '*.json')
json_files = glob.glob(json_pattern)

# JSONファイルが見つからない場合のデフォルト設定
if not json_files:
    default_file = 'disease_categories.json'
    print(f"警告: {json_pattern} でJSONファイルが見つかりませんでした。")
    print(f"デフォルトの '{default_file}' を使用します。")
else:
    # パスを相対パスに変換（表示用）
    relative_paths = [os.path.relpath(p, os.getcwd()) for p in json_files]
    # 省略表示用（表示が長すぎる場合）
    display_paths = [p if len(p) < 50 else f"...{p[-47:]}" for p in relative_paths]
    
    # デフォルトファイルを設定
    default_file = json_files[0]

# ファイル選択のドロップダウン
file_dropdown = widgets.Dropdown(
    options=list(zip(display_paths, json_files)) if json_files else [('デフォルト', default_file)],
    description='JSONファイル:',
    style={'description_width': 'initial'}
)

# 読み込みボタン
load_button = widgets.Button(
    description='読み込み',
    button_style='primary',
    icon='file-import'
)

# カテゴリマネージャーのインスタンス
manager = None

# カテゴリ選択ドロップダウン
category_dropdown = widgets.Dropdown(
    description='カテゴリ:',
    style={'description_width': 'initial'}
)

# ドロップダウン更新関数
def update_category_dropdown():
    if manager is None:
        return
    
    # カテゴリのリストを取得
    categories = list(manager.categories.keys())
    
    # ドロップダウンを更新
    category_dropdown.options = categories
    if categories:
        category_dropdown.value = categories[0]

# 読み込みボタンのクリックイベント
def on_load_button_clicked(b):
    global manager
    file_path = file_dropdown.value
    
    with output_area:
        clear_output()
        print(f"ファイル '{file_path}' からカテゴリを読み込んでいます...")
        manager = CategoryManager(file_path)
        stats = manager.get_category_stats()
        print(f"{len(manager.categories)}個のカテゴリを読み込みました。")
        display(stats)
        update_category_dropdown()

load_button.on_click(on_load_button_clicked)

# 出力エリア
output_area = widgets.Output()

# 読み込みUI
load_ui = widgets.HBox([file_dropdown, load_button])
display(load_ui)
display(output_area)

# 保存タブの更新も忘れずに
def update_save_tab():
    if manager is not None:
        save_label.value = f'現在のファイル: {manager.file_path}'

# 保存タブのラベル（後で更新するため変数に保存）
save_label = widgets.Label(f'現在のファイル: {default_file}')

# 保存タブの更新関数をon_load_button_clickedに追加
def on_load_button_clicked_updated(b):
    global manager
    file_path = file_dropdown.value
    
    with output_area:
        clear_output()
        print(f"ファイル '{file_path}' からカテゴリを読み込んでいます...")
        manager = CategoryManager(file_path)
        stats = manager.get_category_stats()
        print(f"{len(manager.categories)}個のカテゴリを読み込みました。")
        display(stats)
        update_category_dropdown()
        update_save_tab()  # 保存タブも更新

# 更新した関数を登録
load_button.on_click(on_load_button_clicked_updated)

Output()

## カテゴリ管理UI

カテゴリの一覧表示、追加、編集、削除などの機能を提供するUIを作成します。

In [4]:
import os
import glob
from pathlib import Path
import ipywidgets as widgets
from IPython.display import clear_output, display
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import japanize_matplotlib
import re
import json

# リポジトリルートを検出
def find_repo_root():
    current_dir = os.getcwd()
    notebook_dir = Path(current_dir)
    
    # カレントディレクトリがnotebookディレクトリなら
    if notebook_dir.name == 'notebook':
        return str(notebook_dir.parent)
    
    # それ以外の場合は親ディレクトリを探索
    potential_repo_root = notebook_dir
    for _ in range(3):  # 最大3階層まで探索
        if (potential_repo_root / 'src' / 'bin').exists():
            return str(potential_repo_root)
        potential_repo_root = potential_repo_root.parent
    
    # 見つからない場合はカレントディレクトリを返す
    return current_dir

# リポジトリルートを取得
repo_root = find_repo_root()
json_pattern = os.path.join(repo_root, 'src', 'bin', '*.json')
json_files = glob.glob(json_pattern)

# JSONファイルが見つからない場合のデフォルト設定
if not json_files:
    default_file = os.path.join(repo_root, 'src', 'bin', 'disease_categories.json')
    print(f"警告: {json_pattern} でJSONファイルが見つかりませんでした。")
    print(f"デフォルトの '{default_file}' を使用します。")
else:
    # パスを相対パスに変換（表示用）
    relative_paths = [os.path.relpath(p, os.getcwd()) for p in json_files]
    # 表示用の短縮パス
    display_paths = [os.path.basename(p) for p in json_files]
    
    # デフォルトファイルを設定
    default_file = json_files[0]


# ファイル選択のドロップダウン
file_dropdown = widgets.Dropdown(
    options=list(zip(display_paths, json_files)) if json_files else [('デフォルト', default_file)],
    description='JSONファイル:',
    style={'description_width': 'initial'}
)

# カテゴリマネージャーのインスタンス
manager = None

# カテゴリ選択ドロップダウン
category_dropdown = widgets.Dropdown(
    description='カテゴリ:',
    style={'description_width': 'initial'}
)

# ドロップダウン更新関数
def update_category_dropdown():
    if manager is None:
        return
    
    # カテゴリのリストを取得
    categories = list(manager.categories.keys())
    
    # ドロップダウンを更新
    category_dropdown.options = categories
    if categories:
        category_dropdown.value = categories[0]

# 読み込みボタン
load_button = widgets.Button(
    description='読み込み',
    button_style='primary',
    icon='file-import'
)

# 保存タブのラベル
save_label = widgets.Label(f'現在のファイル: {os.path.basename(default_file)}')

# 保存タブの更新関数
def update_save_tab():
    if manager is not None:
        save_label.value = f'現在のファイル: {os.path.basename(manager.file_path)}'

# 出力エリア
output_area = widgets.Output()

# 読み込みUI
load_ui = widgets.HBox([file_dropdown, load_button])

# カテゴリ選択用チェックボックスを動的に生成するコンテナ
category_checks_container = widgets.VBox([])

# 表示ボタン
show_terms_button = widgets.Button(
    description='語句を表示',
    button_style='primary',
    icon='eye'
)

# 全選択/解除ボタン
select_all_button = widgets.Button(
    description='全て選択',
    button_style='info',
    icon='check-square'
)

deselect_all_button = widgets.Button(
    description='全て解除',
    button_style='warning',
    icon='square'
)

# 語句表示エリア
terms_output = widgets.Output()

# チェックボックスを更新する関数
def update_category_checkboxes():
    if manager is None:
        return
    
    # チェックボックスのリストをクリア
    category_checks_container.children = ()
    
    # カテゴリ名でソートしたリスト
    categories = sorted(manager.categories.keys())
    
    # カテゴリごとに語句数を取得
    category_counts = {}
    for category in categories:
        terms = manager.get_category_terms(category)
        category_counts[category] = len(terms)
    
    # 各カテゴリのチェックボックスを生成
    checkboxes = []
    for category in categories:
        checkbox = widgets.Checkbox(
            value=False,
            description=f"{category} ({category_counts[category]}語)",
            indent=False
        )
        checkboxes.append(checkbox)
    
    # チェックボックスを3列で表示するレイアウト
    rows = []
    row_size = 3  # 1行あたりのチェックボックス数
    
    for i in range(0, len(checkboxes), row_size):
        row_boxes = checkboxes[i:i+row_size]
        row = widgets.HBox(row_boxes)
        rows.append(row)
    
    category_checks_container.children = tuple(rows)

# 全選択ボタンのクリックイベント
def on_select_all_button_clicked(b):
    for row in category_checks_container.children:
        for checkbox in row.children:
            checkbox.value = True

# 全解除ボタンのクリックイベント
def on_deselect_all_button_clicked(b):
    for row in category_checks_container.children:
        for checkbox in row.children:
            checkbox.value = False

# 表示ボタンのクリックイベント
def on_show_terms_button_clicked(b):
    if manager is None:
        return
    
    # 選択されたカテゴリを取得
    selected_categories = []
    
    for row in category_checks_container.children:
        for checkbox in row.children:
            if checkbox.value:
                # チェックボックスのラベルからカテゴリ名を抽出
                category = checkbox.description.split(' (')[0]
                selected_categories.append(category)
    
    with terms_output:
        clear_output()
        
        if not selected_categories:
            print("カテゴリが選択されていません。")
            return
        
        print(f"選択された {len(selected_categories)} 個のカテゴリの語句一覧:")
        
        for category in selected_categories:
            if category not in manager.categories:
                continue
                
            terms = manager.get_category_terms(category)
            print(f"\n## {category} ({len(terms)}語)")
            
            # 折り返し表示のためにHTMLを使用
            html_content = "<div style='word-wrap: break-word; white-space: normal; width: 100%;'>"
            
            # すべての語句をカンマ区切りで追加
            term_list = "、".join(terms)
            html_content += term_list
            html_content += "</div>"
            
            # HTML形式で表示
            display(widgets.HTML(html_content))

# ボタンにイベントハンドラを設定
show_terms_button.on_click(on_show_terms_button_clicked)
select_all_button.on_click(on_select_all_button_clicked)
deselect_all_button.on_click(on_deselect_all_button_clicked)

# 読み込みボタンのクリックイベント（更新）
def on_load_button_clicked(b):
    global manager
    file_path = file_dropdown.value
    
    with output_area:
        clear_output()
        print(f"ファイル '{file_path}' からカテゴリを読み込んでいます...")
        manager = CategoryManager(file_path)
        
        # カテゴリ数を表示
        categories_count = len(manager.categories)
        print(f"{categories_count}個のカテゴリを読み込みました。")
        
        # 語句の総数を計算
        total_terms = 0
        for pattern in manager.categories.values():
            terms = pattern.strip("()").split("|")
            total_terms += len(terms)
        
        print(f"登録されている語句の総数: {total_terms}語")
        
        # 各カテゴリの語句数の統計
        print("\n=== カテゴリ別の語句数 ===")
        category_stats = []
        for category_name, pattern in manager.categories.items():
            terms_count = len(pattern.strip("()").split("|"))
            category_stats.append({'カテゴリ': category_name, '語句数': terms_count})
        
        # 語句数でソート
        df_stats = pd.DataFrame(category_stats).sort_values('語句数', ascending=False)
        display(df_stats)
        
        # 各種UIを更新
        update_category_dropdown()
        update_category_checkboxes()  # チェックボックスを更新
        update_save_tab()

# チェックボックス選択UIの作成
category_selector_ui = widgets.VBox([
    widgets.HTML("<h3>カテゴリ別語句一覧</h3>"),
    widgets.HBox([select_all_button, deselect_all_button]),
    category_checks_container,
    show_terms_button,
    terms_output
])

# タブUI
tab = widgets.Tab()

# ===== タブ1: カテゴリ一覧 =====
list_output = widgets.Output()

# 更新ボタン
refresh_button = widgets.Button(
    description='更新',
    button_style='info',
    icon='refresh'
)

def on_refresh_button_clicked(b):
    if manager is None:
        return
    
    with list_output:
        clear_output()
        stats = manager.get_category_stats()
        stats = stats.sort_values('語句数', ascending=False)
        display(stats)
        
        # 可視化
        plt.figure(figsize=(10, 8))
        sns.barplot(x='語句数', y='カテゴリ', data=stats)
        plt.title('カテゴリ別の語句数')
        plt.tight_layout()
        plt.show()

refresh_button.on_click(on_refresh_button_clicked)

list_tab = widgets.VBox([refresh_button, list_output])

# ===== タブ2: カテゴリ追加 =====
# 入力フィールド
new_category_name = widgets.Text(
    description='カテゴリ名:',
    style={'description_width': 'initial'}
)

new_category_pattern = widgets.Textarea(
    description='パターン:',
    placeholder='例: (風邪|感冒|発熱)',
    style={'description_width': 'initial'}
)

# 追加ボタン
add_button = widgets.Button(
    description='追加',
    button_style='success',
    icon='plus'
)

add_output = widgets.Output()

def on_add_button_clicked(b):
    if manager is None:
        return
    
    name = new_category_name.value.strip()
    pattern = new_category_pattern.value.strip()
    
    if not name or not pattern:
        with add_output:
            clear_output()
            print("カテゴリ名とパターンを入力してください。")
        return
    
    with add_output:
        clear_output()
        
        # カテゴリが既に存在するかチェック
        if name in manager.categories:
            print(f"カテゴリ '{name}' は既に存在します。上書きしますか？")
            
            # 上書き確認ボタン
            confirm_button = widgets.Button(
                description='上書きする',
                button_style='warning',
                icon='check'
            )
            
            cancel_button = widgets.Button(
                description='キャンセル',
                button_style='danger',
                icon='times'
            )
            
            def on_confirm(b):
                success = manager.add_category(name, pattern)
                clear_output()
                if success:
                    print(f"カテゴリ '{name}' を上書きしました。")
                    update_category_dropdown()
                else:
                    print("カテゴリの追加に失敗しました。")
            
            def on_cancel(b):
                clear_output()
                print("上書きをキャンセルしました。")
            
            confirm_button.on_click(on_confirm)
            cancel_button.on_click(on_cancel)
            
            display(widgets.HBox([confirm_button, cancel_button]))
        else:
            # 新規追加
            success = manager.add_category(name, pattern)
            if success:
                print(f"カテゴリ '{name}' を追加しました。")
                update_category_dropdown()
            else:
                print("カテゴリの追加に失敗しました。")

add_button.on_click(on_add_button_clicked)

add_tab = widgets.VBox([new_category_name, new_category_pattern, add_button, add_output])

# ===== タブ3: カテゴリ編集 =====
# 語句一覧表示エリア（常に表示）
terms_list_output = widgets.Output()

# 入力フィールド
edit_category_name = widgets.Text(
    description='新カテゴリ名:',
    style={'description_width': 'initial'}
)

edit_terms_textarea = widgets.Textarea(
    description='追加する語句:',
    placeholder='例: 喘鳴|鼻汁|鼻閉|くしゃみ（|で区切って入力）',
    style={'description_width': 'initial'}
)

edit_category_pattern = widgets.Textarea(
    description='現在のパターン:',
    disabled=True,  # 編集不可に設定
    style={'description_width': 'initial'}
)

# 読み込みボタン
load_edit_button = widgets.Button(
    description='読み込み',
    button_style='info',
    icon='download'
)

# 更新ボタン
update_name_button = widgets.Button(
    description='カテゴリ名変更',
    button_style='warning',
    icon='edit'
)

# 語句追加ボタン
add_terms_button = widgets.Button(
    description='語句追加',
    button_style='success',
    icon='plus'
)

# パターン直接編集ボタン
edit_pattern_button = widgets.Button(
    description='パターン編集モード',
    button_style='danger',
    icon='code'
)

edit_output = widgets.Output()

# 語句一覧を更新する関数
def update_terms_list():
    if manager is None or not category_dropdown.value:
        return
    
    category = category_dropdown.value
    
    with terms_list_output:
        clear_output()
        
        # 語句の一覧を表示（元の順序を維持）
        terms = manager.get_category_terms(category)
        print(f"登録されている語句数: {len(terms)}")
        
        if len(terms) > 0:
            # 語句をHTMLで折り返し表示（順序を維持）
            html_content = "<div style='word-wrap: break-word; white-space: normal; width: 100%; margin-top: 10px;'>"
            term_list = "、".join(terms)  # 元の順序のまま結合
            html_content += term_list
            html_content += "</div>"
            
            display(widgets.HTML(html_content))

# 「その他」の統計情報を表示するエリア
others_stats_output = widgets.Output()

# 「その他」の統計情報を更新する関数
def update_others_stats():
    if manager is None:
        return
    
    with others_stats_output:
        clear_output()
        
        try:
            # CSVからデータを読み込み（すでに読み込まれている場合はそれを使用）
            if hasattr(manager, 'data_df') and manager.data_df is not None:
                df = manager.data_df
            else:
                # 最後に処理したCSVファイルのパスを取得
                if hasattr(manager, 'last_csv_path') and manager.last_csv_path:
                    csv_path = manager.last_csv_path
                    column = manager.last_column_name
                else:
                    # CSVがまだ処理されていない場合は統計を表示しない
                    print("CSV分類データがありません。バッチ処理タブでCSVを分類すると表示されます。")
                    return
                
                print(f"CSVファイル '{os.path.basename(csv_path)}' から統計情報を更新しています...")
                df = pd.read_csv(csv_path, encoding='utf-8')
                
                # 前処理
                def preprocess_disease_name(name):
                    if pd.isna(name):
                        return "不明"
                    if isinstance(name, (int, float)):
                        return str(int(name))
                    return str(name)
                
                df[column] = df[column].apply(preprocess_disease_name)
                
                # 分類
                df['normalized_name'] = df[column].apply(manager.normalize_disease_name)
                df['category'] = df['normalized_name'].apply(manager.classify)
                
                # データフレームを保持
                manager.data_df = df
            
            # 「その他」に分類された項目を集計
            others = df[df['category'] == 'その他']
            others_count = len(others)
            total_count = len(df)
            others_percentage = (others_count / total_count) * 100 if total_count > 0 else 0
            
            # ユニークな値の数を取得
            unique_others = others[column].nunique()
            
            print(f"「その他」に分類: {unique_others:,}種類の語句 / {others_count:,}件")
            print(f"全体に対する割合: {others_percentage:.2f}%")
            
            # 「その他」に分類された上位の例を表示
            if unique_others > 0:
                print("\n最も多い「その他」の例:")
                top_others = others[column].value_counts().head(5)
                for term, count in top_others.items():
                    print(f"  {term}: {count:,}件")
        except Exception as e:
            print(f"統計情報の更新中にエラーが発生しました: {e}")
            import traceback
            traceback.print_exc()
            
def on_load_edit_button_clicked(b):
    if manager is None or not category_dropdown.value:
        return
    
    category = category_dropdown.value
    pattern = manager.categories.get(category, "")
    
    edit_category_name.value = category
    edit_category_pattern.value = pattern
    edit_terms_textarea.value = ""  # 語句入力欄をクリア
    
    # 語句一覧を更新
    update_terms_list()
    
    with edit_output:
        clear_output()
        print(f"カテゴリ '{category}' を読み込みました。")

def on_update_name_button_clicked(b):
    if manager is None or not category_dropdown.value:
        return
    
    old_name = category_dropdown.value
    new_name = edit_category_name.value.strip()
    
    if not new_name:
        with edit_output:
            clear_output()
            print("新しいカテゴリ名を入力してください。")
        return
    
    with edit_output:
        clear_output()
        # パターンは変更せず、名前のみ変更
        success = manager.update_category(old_name, new_name)
        if success:
            print(f"カテゴリ名を更新しました: '{old_name}' → '{new_name}'")
            update_category_dropdown()
            # ドロップダウンの選択を新しいカテゴリ名に変更
            if new_name in category_dropdown.options:
                category_dropdown.value = new_name
                # 語句一覧を更新
                update_terms_list()
        else:
            print("カテゴリ名の更新に失敗しました。")

def on_add_terms_edit_button_clicked(b):
    if manager is None or not category_dropdown.value:
        return
    
    category = category_dropdown.value
    terms = edit_terms_textarea.value.strip()
    
    if not terms:
        with edit_output:
            clear_output()
            print("追加する語句を入力してください。")
        return
    
    with edit_output:
        clear_output()
        success = manager.add_terms_to_category(category, terms)
        if success:
            # パターンを更新
            edit_category_pattern.value = manager.categories[category]
            
            print(f"カテゴリ '{category}' に語句を追加しました。")
            
            # 追加した語句を特定
            new_terms = terms.split('|')
            
            print("追加した語句:")
            print("  " + "、".join(new_terms))
            
            # 語句一覧を更新
            update_terms_list()
            
            # 入力欄をクリア
            edit_terms_textarea.value = ""
        else:
            print("語句の追加に失敗しました。")

def on_edit_pattern_button_clicked(b):
    # パターン編集モードの切り替え
    edit_category_pattern.disabled = not edit_category_pattern.disabled
    
    if edit_category_pattern.disabled:
        edit_pattern_button.description = "パターン編集モード"
        edit_pattern_button.button_style = "danger"
    else:
        edit_pattern_button.description = "編集モード終了"
        edit_pattern_button.button_style = "warning"
        
        with edit_output:
            clear_output()
            print("⚠️ 警告: パターン直接編集モードです。正規表現の構文に注意してください。")
            print("パターンの基本構造は (term1|term2|...) の形式である必要があります。")

def on_update_pattern_button_clicked(b):
    if manager is None or not category_dropdown.value or edit_category_pattern.disabled:
        return
        
    category = category_dropdown.value
    new_pattern = edit_category_pattern.value.strip()
    
    if not new_pattern:
        with edit_output:
            clear_output()
            print("パターンを入力してください。")
        return
    
    with edit_output:
        clear_output()
        success = manager.update_category(category, category, new_pattern)
        if success:
            print(f"カテゴリ '{category}' のパターンを更新しました。")
            edit_category_pattern.disabled = True
            edit_pattern_button.description = "パターン編集モード"
            edit_pattern_button.button_style = "danger"
            
            # 語句一覧を更新
            update_terms_list()
        else:
            print("パターンの更新に失敗しました。")

# カテゴリドロップダウンの変更を監視
def on_category_change(change):
    if change['new'] and manager is not None:
        # カテゴリが変更されたら語句一覧を更新
        update_terms_list()

category_dropdown.observe(on_category_change, names='value')

load_edit_button.on_click(on_load_edit_button_clicked)
update_name_button.on_click(on_update_name_button_clicked)
add_terms_button.on_click(on_add_terms_edit_button_clicked)
edit_pattern_button.on_click(on_edit_pattern_button_clicked)

# パターン更新ボタンはパターン編集モード時のみ表示するため、別途定義
update_pattern_button = widgets.Button(
    description='パターン更新',
    button_style='danger',
    icon='save'
)
update_pattern_button.on_click(on_update_pattern_button_clicked)

# 編集モード用のObserver
def on_pattern_edit_mode_change(change):
    if change['new']:  # 編集モード有効
        pattern_edit_controls.children = [edit_pattern_button, update_pattern_button]
    else:  # 編集モード無効
        pattern_edit_controls.children = [edit_pattern_button]

# パターン編集関連コントロール
pattern_edit_controls = widgets.HBox([edit_pattern_button])

# 編集モード監視
edit_category_pattern.observe(
    lambda change: on_pattern_edit_mode_change({'new': not edit_category_pattern.disabled}), 
    names='disabled'
)

edit_tab = widgets.VBox([
    category_dropdown, 
    load_edit_button, 
    widgets.HTML("<h4>選択したカテゴリの登録語句</h4>"),
    terms_list_output,  # 語句一覧を常に表示
    widgets.HTML("<h4>「その他」に分類される語句の統計</h4>"),
    others_stats_output,  # 「その他」の統計情報
    widgets.HTML("<hr>"),
    widgets.HTML("<h4>語句の追加</h4>"),
    edit_terms_textarea,
    add_terms_button,
    widgets.HTML("<hr>"),
    widgets.HTML("<h4>パターンの直接編集（上級者向け）</h4>"),
    edit_category_pattern,
    pattern_edit_controls,
    widgets.HTML("<hr>"),
    widgets.HTML("<h4>カテゴリ名の変更</h4>"),
    edit_category_name, 
    update_name_button,
    edit_output
])
# ===== タブ4: カテゴリ削除 =====
# 削除ボタン
delete_button = widgets.Button(
    description='削除',
    button_style='danger',
    icon='trash'
)

delete_output = widgets.Output()

def on_delete_button_clicked(b):
    if manager is None or not category_dropdown.value:
        return
    
    category = category_dropdown.value
    
    with delete_output:
        clear_output()
        print(f"カテゴリ '{category}' を削除しますか？")
        
        # 確認ボタン
        confirm_button = widgets.Button(
            description='削除する',
            button_style='danger',
            icon='check'
        )
        
        cancel_button = widgets.Button(
            description='キャンセル',
            button_style='info',
            icon='times'
        )
        
        def on_confirm(b):
            success = manager.delete_category(category)
            clear_output()
            if success:
                print(f"カテゴリ '{category}' を削除しました。")
                update_category_dropdown()
            else:
                print("カテゴリの削除に失敗しました。")
        
        def on_cancel(b):
            clear_output()
            print("削除をキャンセルしました。")
        
        confirm_button.on_click(on_confirm)
        cancel_button.on_click(on_cancel)
        
        display(widgets.HBox([confirm_button, cancel_button]))

delete_button.on_click(on_delete_button_clicked)

delete_tab = widgets.VBox([
    widgets.Label('削除するカテゴリを選択:'),
    category_dropdown,
    delete_button,
    delete_output
])

# ===== タブ5: 語句追加 =====
# 入力フィールド
add_terms_textarea = widgets.Textarea(
    description='追加する語句:',
    placeholder='例: 喘鳴|鼻汁|鼻閉|くしゃみ（|で区切って入力）',
    style={'description_width': 'initial'}
)

# 追加ボタン
add_terms_button = widgets.Button(
    description='語句追加',
    button_style='success',
    icon='plus'
)

add_terms_output = widgets.Output()

def on_add_terms_button_clicked(b):
    if manager is None or not category_dropdown.value:
        return
    
    category = category_dropdown.value
    terms = add_terms_textarea.value.strip()
    
    if not terms:
        with add_terms_output:
            clear_output()
            print("追加する語句を入力してください。")
        return
    
    with add_terms_output:
        clear_output()
        success = manager.add_terms_to_category(category, terms)
        if success:
            print(f"カテゴリ '{category}' に語句を追加しました。")
            # 追加後の語句一覧を表示
            all_terms = manager.get_category_terms(category)
            print(f"現在の語句数: {len(all_terms)}")
            
            # 追加した語句を特定
            original_terms = set(manager.get_category_terms(category)) - set(terms.split('|'))
            new_terms = set(terms.split('|'))
            
            print("\n追加した語句:")
            print("、".join(new_terms))
        else:
            print("語句の追加に失敗しました。")

add_terms_button.on_click(on_add_terms_button_clicked)

add_terms_tab = widgets.VBox([
    widgets.Label('語句を追加するカテゴリを選択:'),
    category_dropdown,
    add_terms_textarea,
    add_terms_button,
    add_terms_output
])

# ===== タブ6: CSV入出力 =====
# 出力ファイル名
csv_output_file = widgets.Text(
    value='disease_categories.csv',
    description='CSVファイル:',
    style={'description_width': 'initial'}
)

# エクスポートボタン
export_button = widgets.Button(
    description='エクスポート',
    button_style='primary',
    icon='file-export'
)

# インポートボタン
import_button = widgets.Button(
    description='インポート',
    button_style='info',
    icon='file-import'
)

# インポート方法
import_mode = widgets.RadioButtons(
    options=[('新規カテゴリのみ追加', 'add'), ('すべて上書き', 'overwrite')],
    value='add',
    description='インポート方法:',
    style={'description_width': 'initial'}
)

csv_output = widgets.Output()

def on_export_button_clicked(b):
    if manager is None:
        return
    
    file_path = csv_output_file.value.strip()
    if not file_path:
        file_path = 'disease_categories.csv'
    
    with csv_output:
        clear_output()
        try:
            df = manager.export_to_csv(file_path)
            print(f"{len(df)}件のデータをCSVにエクスポートしました。")
            display(df.head(10))
        except Exception as e:
            print(f"エクスポート中にエラーが発生しました: {e}")

def on_import_button_clicked(b):
    if manager is None:
        return
    
    file_path = csv_output_file.value.strip()
    if not file_path:
        with csv_output:
            clear_output()
            print("CSVファイル名を入力してください。")
        return
    
    with csv_output:
        clear_output()
        try:
            overwrite = import_mode.value == 'overwrite'
            success = manager.import_from_csv(file_path, overwrite)
            if success:
                print(f"CSVからカテゴリをインポートしました。")
                update_category_dropdown()
            else:
                print("インポートに失敗しました。")
        except Exception as e:
            print(f"インポート中にエラーが発生しました: {e}")

export_button.on_click(on_export_button_clicked)
import_button.on_click(on_import_button_clicked)

csv_tab = widgets.VBox([
    csv_output_file,
    widgets.HBox([export_button, import_button]),
    import_mode,
    csv_output
])

# ===== タブ7: テスト =====
# テストテキスト入力
test_text = widgets.Text(
    value='',
    placeholder='例: 頭痛、発熱、咳、腹痛など',
    description='テストテキスト:',
    style={'description_width': 'initial'}
)

# テストボタン
test_button = widgets.Button(
    description='テスト',
    button_style='info',
    icon='search'
)

test_output = widgets.Output()

def on_test_button_clicked(b):
    if manager is None:
        return
    
    text = test_text.value.strip()
    if not text:
        with test_output:
            clear_output()
            print("テストするテキストを入力してください。")
        return
    
    with test_output:
        clear_output()
        results = manager.test_pattern(text)
        matched = [cat for cat, match, _ in results if match]
        
        print(f"テキスト: '{text}'")
        
        if matched:
            print(f"\n一致したカテゴリ: {len(matched)}個")
            for cat, match, match_text in results:
                if match:
                    print(f"✓ {cat}: {match_text}")
            
            print(f"\n分類結果: {manager.classify(text)}")
        else:
            print("どのカテゴリにも一致しませんでした。「その他」に分類されます。")

test_button.on_click(on_test_button_clicked)

test_tab = widgets.VBox([
    test_text,
    test_button,
    test_output
])

# ===== タブ8: 保存 =====
# 保存パス入力フィールド - ファイル名のみ入力
save_filename_input = widgets.Text(
    value='',
    placeholder='例: new_categories.json',
    description='ファイル名:',
    style={'description_width': 'initial'}
)

# 通常保存ボタン
save_button = widgets.Button(
    description='保存',
    button_style='success',
    icon='save'
)

# 別名保存ボタン
save_as_button = widgets.Button(
    description='別名で保存',
    button_style='info',
    icon='file-export'
)

save_output = widgets.Output()

def on_save_button_clicked(b):
    if manager is None:
        return
    
    with save_output:
        clear_output()
        try:
            manager.save_categories()
            print(f"{len(manager.categories)}個のカテゴリを '{manager.file_path}' に保存しました。")
        except Exception as e:
            print(f"保存中にエラーが発生しました: {e}")

def on_save_as_button_clicked(b):
    if manager is None:
        return
    
    filename = save_filename_input.value.strip()
    
    with save_output:
        clear_output()
        
        if not filename:
            print("保存先ファイル名を入力してください。")
            return
        
        try:
            # 現在のファイルと同じディレクトリに保存
            current_dir = os.path.dirname(manager.file_path)
            save_path = os.path.join(current_dir, filename)
            
            # 現在のファイルパスを保存
            original_path = manager.file_path
            
            # 一時的にファイルパスを変更して保存
            manager.file_path = save_path
            manager.save_categories()
            
            print(f"{len(manager.categories)}個のカテゴリを '{save_path}' に保存しました。")
            
            # ファイルパスを元に戻すかどうか確認ボタンを表示
            keep_button = widgets.Button(
                description='新しいファイルに切り替え',
                button_style='success',
                icon='check'
            )
            
            revert_button = widgets.Button(
                description='元のファイルに戻す',
                button_style='warning',
                icon='undo'
            )
            
            def on_keep(b):
                # 新しいパスを維持する場合はファイルドロップダウンを更新
                nonlocal save_path
                update_save_tab()
                clear_output()
                print(f"作業中のファイルを '{save_path}' に変更しました。")
                
                # ファイルドロップダウンにない場合は追加
                file_paths = [f[1] for f in file_dropdown.options]
                if save_path not in file_paths:
                    new_options = list(file_dropdown.options)
                    new_options.append((filename, save_path))
                    file_dropdown.options = new_options
                
                # ドロップダウンの選択を変更
                file_dropdown.value = save_path
            
            def on_revert(b):
                # 元のパスに戻す
                nonlocal original_path
                manager.file_path = original_path
                update_save_tab()
                clear_output()
                print(f"別名保存しましたが、作業中のファイルは '{os.path.basename(original_path)}' のままです。")
            
            keep_button.on_click(on_keep)
            revert_button.on_click(on_revert)
            
            print("\n保存後の操作を選択してください:")
            display(widgets.HBox([keep_button, revert_button]))
            
        except Exception as e:
            print(f"保存中にエラーが発生しました: {e}")
            import traceback
            traceback.print_exc()

save_button.on_click(on_save_button_clicked)
save_as_button.on_click(on_save_as_button_clicked)

# 保存タブを修正
save_tab = widgets.VBox([
    save_label,
    save_button,
    widgets.HTML("<hr style='margin: 10px 0;'>"),
    widgets.HTML("<p>同じディレクトリに別名で保存:</p>"),
    save_filename_input,
    save_as_button,
    save_output
])

# タブを設定
tab.children = [list_tab, add_tab, edit_tab, delete_tab, add_terms_tab, csv_tab, test_tab, save_tab]
tab.set_title(0, '一覧')
tab.set_title(1, '追加')
tab.set_title(2, '編集')
tab.set_title(3, '削除')
tab.set_title(4, '語句追加')
tab.set_title(5, 'CSV入出力')
tab.set_title(6, 'テスト')
tab.set_title(7, '保存')

# 最初は一覧タブを表示
tab.selected_index = 0

# display(tab)

# 自動で最初のデータを読み込む
if os.path.exists(file_dropdown.value):
    on_load_button_clicked(None)
    on_refresh_button_clicked(None)

# バッチ処理UI
# CSVファイルの選択
input_csv_file = widgets.Text(
    value='',
    placeholder='例: 傷病名一覧.csv',
    description='入力CSVファイル:',
    style={'description_width': 'initial'}
)

# 傷病名の列名
disease_column = widgets.Text(
    value='傷病名',
    description='傷病名の列:',
    style={'description_width': 'initial'}
)

# 出力CSVファイル名
output_csv_file = widgets.Text(
    value='classified_diseases.csv',
    description='出力CSVファイル:',
    style={'description_width': 'initial'}
)

# 分類実行ボタン
classify_button = widgets.Button(
    description='分類実行',
    button_style='primary',
    icon='play'
)

batch_output = widgets.Output()

def on_classify_button_clicked(b):
    if manager is None:
        with batch_output:
            clear_output()
            print("先にカテゴリを読み込んでください。")
        return
    
    input_file = input_csv_file.value.strip()
    column = disease_column.value.strip()
    output_file = output_csv_file.value.strip()
    
    if not input_file or not output_file or not column:
        with batch_output:
            clear_output()
            print("入力ファイル、出力ファイル、列名を指定してください。")
        return
    
    with batch_output:
        clear_output()
        try:
            print(f"CSVファイル '{input_file}' を読み込んでいます...")
            df = pd.read_csv(input_file, encoding='utf-8')
            
            if column not in df.columns:
                print(f"列 '{column}' がCSVファイルに存在しません。")
                print(f"利用可能な列: {', '.join(df.columns)}")
                return
            
            print(f"{len(df)}件のデータを読み込みました。分類を開始します...")
            
            # 前処理関数
            def preprocess_disease_name(name):
                if pd.isna(name):
                    return "不明"
                if isinstance(name, (int, float)):
                    return str(int(name))
                return str(name)
            
            # 前処理
            df[column] = df[column].apply(preprocess_disease_name)
            
            # 正規化
            df['normalized_name'] = df[column].apply(manager.normalize_disease_name)
            
            # 分類
            df['category'] = df['normalized_name'].apply(manager.classify)
            
            # カテゴリごとの集計
            category_counts = df['category'].value_counts()
            
            # 結果の表示
            print("\n=== 分類結果 ===")
            print("\n各カテゴリの件数:")
            for category, count in category_counts.items():
                print(f"{category}: {count:,}件")
            
            # API呼び出し必要数の計算
            api_calls_needed = int(category_counts.get('その他', 0))
            api_calls_percentage = (api_calls_needed / len(df)) * 100
            
            print(f"\n必要なAPI呼び出し回数: {api_calls_needed:,}回")
            print(f"全サンプルに対する割合: {api_calls_percentage:.2f}%")
            
            # 可視化
            plt.figure(figsize=(12, 8))
            sns.barplot(x=category_counts.values, y=category_counts.index)
            plt.title('傷病名カテゴリ別の件数')
            plt.xlabel('件数')
            plt.ylabel('カテゴリ')
            plt.tight_layout()
            plt.show()
            
            # その他に分類された例の確認
            print("\n=== 「その他」に分類された傷病名の例（先頭20件）===")
            other_examples = df[df['category'] == 'その他'][column].value_counts().head(20)
            for disease, count in other_examples.items():
                print(f"{disease}: {count:,}件")
            
            # CSV出力
            df.to_csv(output_file, encoding='utf-8', index=False)
            print(f"\n分類結果を '{output_file}' に保存しました。")
            
        except Exception as e:
            print(f"エラーが発生しました: {e}")
            import traceback
            traceback.print_exc()
    with batch_output:
        clear_output()
        try:
            print(f"CSVファイル '{input_file}' を読み込んでいます...")
            df = pd.read_csv(input_file, encoding='utf-8')
            
            # 既存の処理...
            
            # CSV出力
            df.to_csv(output_file, encoding='utf-8', index=False)
            print(f"\n分類結果を '{output_file}' に保存しました。")
            
            # 最後に処理したCSVのパスと列名を保存
            manager.last_csv_path = input_file
            manager.last_column_name = column
            manager.data_df = df
            
            # 「その他」の統計情報を更新
            update_others_stats()
            
        except Exception as e:
            print(f"エラーが発生しました: {e}")
            import traceback
            traceback.print_exc()

classify_button.on_click(on_classify_button_clicked)

# バッチ処理UI
batch_ui = widgets.VBox([
    widgets.HTML("<h3>CSVファイルの傷病名を一括分類</h3>"),
    input_csv_file,
    disease_column,
    output_csv_file,
    classify_button,
    batch_output
])
# メインレイアウト - 縦方向に配置
main_layout = widgets.VBox([
    # ファイル選択UI
    widgets.HTML("<h2>カテゴリファイル管理</h2>"),
    load_ui,
    # カテゴリ選択UI
    category_selector_ui,
    
    # タブインターフェース
    widgets.HTML("<h2>カテゴリ編集</h2>"),
    tab,
    
    # バッチ処理UI
    widgets.HTML("<h2>バッチ処理</h2>"),
    batch_ui
])

# バッチ処理は下に配置
batch_section = widgets.VBox([
    widgets.HTML("<h2>バッチ処理</h2>"),
    batch_ui
])

# 全体のレイアウト
display(main_layout)


In [ ]:
update_others_stats()